In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)
 
os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns

import pickle 
import logging


from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning
/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Code


In [3]:
querySno = SQLQuery('snowflake')
ga_df = querySno(r"""
with default_flags as (
select distinct lending_business_id, business_id,loan_month,
max(case when emi_no = 1 then due_date end) over(partition by lending_business_id) as first_duedate, 
max(case when emi_no = 2 then due_date end) over(partition by lending_business_id) as second_duedate, 
max(case when emi_no = 3 then due_date end) over(partition by lending_business_id) as third_duedate,
max(case when emi_no = 4 then due_date end) over(partition by lending_business_id) as fourth_duedate,
max(case when emi_no = 1 then defalt end) over(partition by lending_business_id) as fpd, 
max(case when emi_no = 2 then defalt end) over(partition by lending_business_id) as spd, 
max(case when emi_no = 3 then defalt end) over(partition by lending_business_id) as tpd,
max(case when emi_no = 4 then defalt end) over(partition by lending_business_id) as frpd,
max(case when emi_no = 1 then total_emi end) over(partition by lending_business_id) as first_emi_amount,
max(case when emi_no = 2 then total_emi end) over(partition by lending_business_id) as second_emi_amount,
max(case when emi_no = 3 then total_emi end) over(partition by lending_business_id) as third_emi_amount,
max(case when emi_no = 4 then total_emi end) over(partition by lending_business_id) as fourth_emi_amount
from (
select a.*,json_extract_path_text(c.predict_meta, 'business_id') as business_id, month(a.created_at) as month_no,
case when paid_on_time = 'true' then 0
when paid_on_time = 'false' then 1 else null end as default_flag,
dense_rank() over(partition by a.lending_business_id order by due_date) as emi_no,
min(a.created_at) over (partition by a.lending_business_id) as min_created_at,
month(min_created_at) as loan_month,
max(default_flag) over(partition by a.lending_business_id, due_date) as defalt,
sum(due_amount) over(partition by a.lending_business_id, due_date) as amount_due,
sum(payment) over(partition by a.lending_business_id, due_date) as payment_done,
amount_due/100 + payment_done/100 as total_emi
from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES" a
join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS c on a.lending_business_id = c.lending_business_id
where date(due_date) < date('2023-05-05')
-- and a.lending_business_id = '909564ef-6b22-4303-9df5-fc4e892fef01'
)
)
, first_cure as (
select *,transaction_date as first_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) rnk
from (
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as first_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.second_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where first_amount_cured >= first_emi_amount
)
where rnk = 1
)
, second_cure as (
select *, transaction_date as second_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
t.running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as second_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.second_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where second_amount_cured >= first_emi_amount + second_emi_amount
)
where rnk = 1
)

,third_cure as (
select *, transaction_date as third_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as third_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.third_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where third_amount_cured >= first_emi_amount + second_emi_amount + third_emi_amount
)
where rnk = 1
)

,fourth_cure as (
select *, transaction_date as fourth_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount,b.fourth_emi_amount ,t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,b.fourth_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as fourth_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.third_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where fourth_amount_cured >= first_emi_amount + second_emi_amount + third_emi_amount + fourth_emi_amount
)
where rnk = 1
)

select a.lending_business_id, a.business_id, a.first_duedate,date(f.first_cure_date)as first_cure_date, a.second_duedate,date(s.second_cure_date)as second_cure_date, 
a.third_duedate,date(t.third_cure_date) as third_cure_date, a.fourth_duedate,date(fr.fourth_cure_date) as fourth_cure_date,fpd, spd, tpd, frpd,
case when fpd = 1 and first_amount_cured >= a.first_emi_amount then 1 else 0 end as first_cure_flag,
case when spd = 1 and second_amount_cured >= a.first_emi_amount + a.second_emi_amount then 1 else 0 end as second_cure_flag,
case when tpd = 1 and third_amount_cured >= a.first_emi_amount + a.second_emi_amount + a.third_emi_amount then 1 else 0 end as third_cure_flag,
case when frpd = 1 and fourth_amount_cured >= a.first_emi_amount + a.second_emi_amount + a.third_emi_amount + a.fourth_emi_amount then 1 else 0 end as fourth_cure_flag,
case when first_cure_flag = 1 then datediff(day,a.first_duedate,f.first_cure_date) else null end as first_cure_days,
case when second_cure_flag = 1 then datediff(day,a.second_duedate,s.second_cure_date) else null end as second_cure_days,
case when third_cure_flag = 1 then datediff(day,a.third_duedate,t.third_cure_date) else null end as third_cure_days,
case when fourth_cure_flag = 1 then datediff(day,a.fourth_duedate,fr.fourth_cure_date) else null end as fourth_cure_days,
a.first_emi_amount, f.first_amount_cured, a.second_emi_amount,(s.second_amount_cured - f.first_amount_cured) as second_amt_cured, a.third_emi_amount, 
(t.third_amount_cured - second_amt_cured) as third_amt_cured, a.fourth_emi_amount, (fr.fourth_amount_cured - third_amt_cured) as fourth_amt_cured
from default_flags a 
left join first_cure f on a.lending_business_id = f.lending_business_id
left join second_cure s on a.lending_business_id = s.lending_business_id
left join third_cure t on a.lending_business_id = t.lending_business_id
left join fourth_cure fr on a.lending_business_id = fr.lending_business_id
where a.first_duedate < date('2023-05-19')
-- and a.business_id = '111eb19b-cd06-4943-b9f5-c111ae38189e'
order by first_duedate desc
""")

In [4]:
ga_df.shape

(2229, 30)

In [5]:
ga_df.head()

,lending_business_id,business_id,first_duedate,first_cure_date,second_duedate,second_cure_date,third_duedate,third_cure_date,fourth_duedate,fourth_cure_date,...,third_cure_days,fourth_cure_days,first_emi_amount,first_amount_cured,second_emi_amount,second_amt_cured,third_emi_amount,third_amt_cured,fourth_emi_amount,fourth_amt_cured
0,36938cc2-0100-435f-8f03-4a3a741108c6,1e1e434d-3b59-4ca1-b753-cb4f127cce97,2023-05-04,None,None,None,None,None,None,None,...,NaN,NaN,189.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,76696dc2-3a2e-44c6-9660-45f3f57e3929,ff0aa54b-e339-445e-8e7d-571b43909509,2023-05-04,None,None,None,None,None,None,None,...,NaN,NaN,76.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,89df7cea-d8cf-4060-8eed-057c3717e8c6,bcb6904e-8688-4f2c-89b5-4b480ef6484e,2023-05-04,2023-05-04,None,None,None,None,None,None,...,NaN,NaN,283.75,283.75,NaN,NaN,NaN,NaN,NaN,NaN
3,ab86eb05-b55a-4f5c-9e97-6ea37f369c01,a4c275ca-dfd1-4e09-bfe7-9016e7206514,2023-05-04,None,None,None,None,None,None,None,...,NaN,NaN,1053.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,09e0ed1b-d1f5-4eea-a1aa-547d9a697c67,0e04e84c-edcc-4809-92b3-4e8ba5a8c7e5,2023-05-04,2023-05-04,None,None,None,None,None,None,...,NaN,NaN,2307.16,2307.16,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
ga_df['DPD_plus_15'] = 0
ga_df.loc[((ga_df['fpd']==1) & (ga_df['first_cure_days']>=15))| ((ga_df['spd']==1) & (ga_df['second_cure_days']>=15)) | ((ga_df['tpd']==1) & (ga_df['third_cure_days']>=15)) | ((ga_df['frpd']==1) & (ga_df['fourth_cure_days']>=15)),'DPD_plus_15'] = 1

In [7]:
ga_df['DPD_plus_15'].value_counts()

0    1990
1     239
Name: DPD_plus_15, dtype: int64

In [8]:
ga_df['DPD_plus_30'] = 0
ga_df.loc[((ga_df['fpd']==1) & (ga_df['first_cure_days']>=30))| ((ga_df['spd']==1) & (ga_df['second_cure_days']>=30)) | ((ga_df['tpd']==1) & (ga_df['third_cure_days']>=30)) | ((ga_df['frpd']==1) & (ga_df['fourth_cure_days']>=30)),'DPD_plus_30'] = 1

In [9]:
ga_df['DPD_plus_30'].value_counts()

0    2062
1     167
Name: DPD_plus_30, dtype: int64

In [10]:
ga_df['FPD_plus_3'] = 0
ga_df['SPD_plus_3'] = 0
ga_df['TPD_plus_3'] = 0
ga_df['FRPD_plus_3'] = 0
ga_df.loc[(ga_df['fpd']==1) & (ga_df['first_cure_days']>=3),'FPD_plus_3'] = 1
ga_df.loc[(ga_df['spd']==1) & (ga_df['second_cure_days']>=3),'SPD_plus_3'] = 1
ga_df.loc[(ga_df['tpd']==1) & (ga_df['third_cure_days']>=3),'TPD_plus_3'] = 1
ga_df.loc[(ga_df['frpd']==1) & (ga_df['fourth_cure_days']>=3),'FRPD_plus_3'] = 1

In [11]:
ga_df['FPD_plus_15'] = 0
ga_df['SPD_plus_15'] = 0
ga_df['TPD_plus_15'] = 0
ga_df['FRPD_plus_15'] = 0
ga_df.loc[(ga_df['fpd']==1) & (ga_df['first_cure_days']>=15),'FPD_plus_15'] = 1
ga_df.loc[(ga_df['spd']==1) & (ga_df['second_cure_days']>=15),'SPD_plus_15'] = 1
ga_df.loc[(ga_df['tpd']==1) & (ga_df['third_cure_days']>=15),'TPD_plus_15'] = 1
ga_df.loc[(ga_df['frpd']==1) & (ga_df['fourth_cure_days']>=15),'FRPD_plus_15'] = 1

In [12]:
ga_df['FPD_plus_30'] = 0
ga_df['SPD_plus_30'] = 0
ga_df['TPD_plus_30'] = 0
ga_df['FRPD_plus_30'] = 0
ga_df.loc[(ga_df['fpd']==1) & (ga_df['first_cure_days']>=30),'FPD_plus_30'] = 1
ga_df.loc[(ga_df['spd']==1) & (ga_df['second_cure_days']>=30),'SPD_plus_30'] = 1
ga_df.loc[(ga_df['tpd']==1) & (ga_df['third_cure_days']>=30),'TPD_plus_30'] = 1
ga_df.loc[(ga_df['frpd']==1) & (ga_df['fourth_cure_days']>=30),'FRPD_plus_30'] = 1

In [13]:
ga_first = ga_df[['business_id','first_duedate','FPD_plus_3','FPD_plus_15','FPD_plus_30']].rename(columns={'first_duedate': 'due_date', 'FPD_plus_3': 'DPD_plus_3','FPD_plus_15': 'DPD_plus_15','FPD_plus_30': 'DPD_plus_30'})
ga_second = ga_df[['business_id','second_duedate','SPD_plus_3','SPD_plus_15','SPD_plus_30']].rename(columns={'second_duedate': 'due_date', 'SPD_plus_3': 'DPD_plus_3','SPD_plus_15': 'DPD_plus_15','SPD_plus_30': 'DPD_plus_30'})
ga_third = ga_df[['business_id','third_duedate','TPD_plus_3','TPD_plus_15','TPD_plus_30']].rename(columns={'third_duedate': 'due_date', 'TPD_plus_3': 'DPD_plus_3','TPD_plus_15': 'DPD_plus_15','TPD_plus_30': 'DPD_plus_30'})
ga_fourth = ga_df[['business_id','fourth_duedate','FRPD_plus_3','FRPD_plus_15','FRPD_plus_30']].rename(columns={'fourth_duedate': 'due_date', 'FRPD_plus_3': 'DPD_plus_3','FRPD_plus_15': 'DPD_plus_15','FRPD_plus_30': 'DPD_plus_30'})

In [14]:
ga_first['emi_no'] = 1
ga_second['emi_no'] = 2
ga_third['emi_no'] = 3
ga_fourth['emi_no'] = 4

In [15]:
ga_stack = pd.concat([ga_first, ga_second, ga_third, ga_fourth], axis = 0, ignore_index = True)

In [16]:
ga_stack.shape

(8916, 6)

In [17]:
ga_first.shape

(2229, 6)

In [18]:
ga_fourth.sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
127,d26efcc9-2a8b-4886-80b8-6b86f683d40b,None,0,0,0,4
1589,3be4a387-b7b3-43c8-b25f-d64f7a8b0ffa,None,0,0,0,4
721,d4804714-fd24-481a-bd5d-68ebc06ac6a7,None,0,0,0,4
104,2a49fdec-5fdc-445a-a7b7-a845ba91f10c,None,0,0,0,4
1911,0202e995-8239-4235-8099-bb0bee709bd6,2023-04-29,0,0,0,4


In [19]:
ga_stack[ga_stack['due_date'].isnull()].shape

(3599, 6)

In [20]:
ga_stack = ga_stack[~ga_stack['due_date'].isnull()]

In [21]:
ga_stack.shape

(5317, 6)

In [22]:
ga_stack['emi_no'].value_counts()

1    2229
2    1667
3    1075
4     346
Name: emi_no, dtype: int64

In [23]:
df = ga_stack.copy()

In [24]:
df['DPD_plus_15'].value_counts()

0    5034
1     283
Name: DPD_plus_15, dtype: int64

In [25]:
df.head()

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
0,1e1e434d-3b59-4ca1-b753-cb4f127cce97,2023-05-04,0,0,0,1
1,ff0aa54b-e339-445e-8e7d-571b43909509,2023-05-04,0,0,0,1
2,bcb6904e-8688-4f2c-89b5-4b480ef6484e,2023-05-04,0,0,0,1
3,a4c275ca-dfd1-4e09-bfe7-9016e7206514,2023-05-04,0,0,0,1
4,0e04e84c-edcc-4809-92b3-4e8ba5a8c7e5,2023-05-04,0,0,0,1


In [26]:
df.shape

(5317, 6)

In [27]:
df = df.drop_duplicates(subset=['business_id','due_date'])

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5314 entries, 0 to 8915
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  5314 non-null   object
 1   due_date     5314 non-null   object
 2   DPD_plus_3   5314 non-null   int64 
 3   DPD_plus_15  5314 non-null   int64 
 4   DPD_plus_30  5314 non-null   int64 
 5   emi_no       5314 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 290.6+ KB


In [29]:
df["due_date"] = df["due_date"].astype('datetime64[ns]')

In [30]:
df['emi_no'].value_counts()

1    2228
2    1666
3    1074
4     346
Name: emi_no, dtype: int64

In [31]:
# sdf= querySno(r""" 
# select * from PROD_DB.ADHOC.REDEC_SCORED_DATASET
# """)

In [32]:
# sdf = pd.read_pickle(f'/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/data/new_dataset_scored.pkl')

In [33]:
# sdf.shape

In [34]:
# sdf.info()

In [35]:
# sdf["due_date"] = sdf["due_date"].astype('datetime64[ns]')

In [36]:
# sdf.head()

In [37]:
# sdf = sdf[['business_id','due_date', 'emi_no', 'redec_proba','redec_score','redec_bin']]

In [38]:
# df = df.merge(sdf,left_on = ['business_id','due_date'],right_on = ['business_id','due_date'])

In [39]:
df.shape

(5314, 6)

In [40]:
df.head()

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
0,1e1e434d-3b59-4ca1-b753-cb4f127cce97,2023-05-04,0,0,0,1
1,ff0aa54b-e339-445e-8e7d-571b43909509,2023-05-04,0,0,0,1
2,bcb6904e-8688-4f2c-89b5-4b480ef6484e,2023-05-04,0,0,0,1
3,a4c275ca-dfd1-4e09-bfe7-9016e7206514,2023-05-04,0,0,0,1
4,0e04e84c-edcc-4809-92b3-4e8ba5a8c7e5,2023-05-04,0,0,0,1


In [41]:
df = df.merge(df.sort_values(['business_id','emi_no']).groupby(['business_id'])['DPD_plus_15'].shift(-1).rename('DPD_plus_15_next'),left_index=True,right_index=True)

In [42]:
df.head()

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no,DPD_plus_15_next
0,1e1e434d-3b59-4ca1-b753-cb4f127cce97,2023-05-04,0,0,0,1,NaN
1,ff0aa54b-e339-445e-8e7d-571b43909509,2023-05-04,0,0,0,1,NaN
2,bcb6904e-8688-4f2c-89b5-4b480ef6484e,2023-05-04,0,0,0,1,NaN
3,a4c275ca-dfd1-4e09-bfe7-9016e7206514,2023-05-04,0,0,0,1,NaN
4,0e04e84c-edcc-4809-92b3-4e8ba5a8c7e5,2023-05-04,0,0,0,1,NaN


In [43]:
df[df['emi_no']==4].sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no,DPD_plus_15_next
8700,eebd7d61-1061-4ae6-a77e-1cd49e865415,2023-04-08,0,0,0,4,NaN
8596,2607470d-42e2-4c42-be9d-56f9b8869399,2023-04-30,0,0,0,4,NaN
8592,867f4b74-1783-4b2b-ba75-7197e3089ad9,2023-04-30,0,0,0,4,NaN
8822,19b10407-d2ea-4aee-8256-565b2a060d35,2023-04-01,0,0,0,4,NaN
8790,dd02cea4-a9a8-4ee0-9682-86b8f86a5a7d,2023-04-02,0,0,0,4,NaN


In [44]:
df = df.fillna(0)

In [45]:
ddf = querySno(r"""
select json_extract_path_text(predict_meta, 'business_id') as business_id, json_extract_path_text(predict_meta, 'default_bin') as rs2_bin , novo_risk_score as rs2_score , fico_score
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS a 
left join  "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."CUSTOMER_CREDIT_PROFILES" b 
on a.lending_business_id = b.lending_business_id
""")

In [46]:
ddf.head()

,business_id,rs2_bin,rs2_score,fico_score
0,25539d6a-d201-4564-a6a6-b30be4ab5632,5,883.0,794
1,919f8399-5794-45c0-8ecc-cc693469d487,3,544.0,625
2,90524c16-28c7-4fd7-8aa8-48eb47d4ea78,4,673.0,475
3,6d3acc5f-effb-4685-9f43-529a308b14e5,5,820.0,721
4,f35df4ad-9121-4d2d-8075-bdf967c2bd58,3,551.0,740


In [47]:
ddf.shape

(9635, 4)

In [48]:
ddf = ddf.drop_duplicates(subset=['business_id'])

In [49]:
ddf2 = querySno(r""" 
select *, novo_risk_score as rs1_score
from (
select *, row_number() over(partition by business_id order by offer_start_at desc) rnk from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_INVITATIONS"
)
where rnk = 1

""")

In [50]:
ddf2.head()

,reason,apr,novo_response_score,offer_amount,offer_start_at,novo_risk_score,created_at,is_amount_drawn,lending_invitation_file_id,offer_end_at,...,test_key,id,business_id,campaign_id,offer_rejected_at,status,_fivetran_deleted,_fivetran_synced,rnk,rs1_score
0,Application denied - Final Decision (DENIED),36,779.3468979,920000,2023-01-10,869.1205911,2023-01-10 16:52:04.073000+00:00,False,68fbcaa4-8fab-42dc-88f0-5949a5ebc248,2023-02-10,...,NA,14e86529-d1f0-4603-96c8-78cfa71dc751,7a5b422d-0ee6-4fec-9a34-e49124ab04a9,2,2023-01-10,rejected,None,2023-05-06 03:32:15.501000+00:00,1,869.1205911
1,N/A,36,373.286189,180000,2023-04-25,582.8024286,2023-04-25 08:44:51.613000+00:00,False,941d9b97-3f27-4bfa-8d63-079bae9511ca,2023-05-25,...,NA,4f26dc61-6f7f-44c8-a305-094dcd4fa9d1,b55322f3-03e7-43c2-9178-a06e8b331133,46,None,active,None,2023-05-06 03:32:18.005000+00:00,1,582.8024286
2,Invitation Expired by the system,36,365.7594767,560000,2023-04-12,978.0460709,2023-04-12 11:28:41.368000+00:00,False,761ac08a-c4da-4fcd-b3f3-16226a37bb5f,2023-05-12,...,NA,649364a9-5e02-42e7-82e3-fb387c41e7ae,919bc116-3ae1-44e5-b7dd-a2ffe0315241,40,None,expired,None,2023-05-13 07:12:40.220000+00:00,1,978.0460709
3,Application denied - Final Decision (DENIED),36,668.4422225,590000,2023-02-03,844.4072022,2023-02-03 14:26:50.833000+00:00,False,3bcca038-7077-4642-a6e7-9bc9b87bb848,2023-03-05,...,NA,cda26e01-ff51-4229-9a4c-3a6a0c3848b6,a68d9411-c07f-47a8-a347-b6fd80d815b2,12,2023-02-10,rejected,None,2023-05-06 03:32:16.110000+00:00,1,844.4072022
4,"Invalid Data: START_DATE, END_DATE,",36,507.7478426,1000000,None,718.2779645,2023-03-13 14:13:11.215000+00:00,False,1043ecfd-dda8-446f-b528-2613d365acb8,None,...,Etsy,648822f9-c776-4891-a244-e8b9c0ff5dea,2c1d6518-1f35-4de0-8269-941388381acc,27,None,error,None,2023-05-06 03:32:16.340000+00:00,1,718.2779645


In [51]:
ddf2 = ddf2[['business_id','rs1_score']]

In [52]:
def rs1_bin(x):
    try: x = float(x)
    except: x = np.nan
    if x >= 656:
        return 5
    if x >= 545:
        return 4
    if x >= 489:
        return 3
    if x >= 443:
        return 2
    return 1

In [53]:
ddf2['rs1_bin'] = ddf2['rs1_score'].apply(lambda x: rs1_bin(x))

In [54]:
ddf2['rs1_bin'].value_counts()

5    36674
4     8050
3      451
1      272
2      263
Name: rs1_bin, dtype: int64

In [55]:
df0 = df.merge(ddf,how='left',left_on = 'business_id',right_on='business_id')
rdf = df0.merge(ddf2,how='left',left_on = 'business_id',right_on='business_id')

In [56]:
rdf.head()

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no,DPD_plus_15_next,rs2_bin,rs2_score,fico_score,rs1_score,rs1_bin
0,1e1e434d-3b59-4ca1-b753-cb4f127cce97,2023-05-04,0,0,0,1,0.0,4,613.0,740,633.219822,4
1,ff0aa54b-e339-445e-8e7d-571b43909509,2023-05-04,0,0,0,1,0.0,3,599.0,772,726.8394453,5
2,bcb6904e-8688-4f2c-89b5-4b480ef6484e,2023-05-04,0,0,0,1,0.0,4,655.0,780,698.4256213,5
3,a4c275ca-dfd1-4e09-bfe7-9016e7206514,2023-05-04,0,0,0,1,0.0,5,818.0,801,885.4446325,5
4,0e04e84c-edcc-4809-92b3-4e8ba5a8c7e5,2023-05-04,0,0,0,1,0.0,5,998.0,737,925.3730837,5


In [57]:
rdf.shape

(5314, 12)

In [58]:
rdf['rs1_bin'].value_counts()

5    4636
4     368
1     181
3      92
2      37
Name: rs1_bin, dtype: int64

In [59]:
rdf['rs2_bin'].value_counts()

5    2119
3    1354
4    1215
2     395
1     231
Name: rs2_bin, dtype: int64

In [60]:
rdf['emi_no'].value_counts()

1    2228
2    1666
3    1074
4     346
Name: emi_no, dtype: int64

In [61]:
# rdf.to_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/data/new_DPD_dataset.pkl')

In [62]:
pd.crosstab(rdf['rs1_bin'],rdf['DPD_plus_15_next']).to_clipboard()

In [63]:
pd.crosstab(rdf['rs2_bin'],rdf['DPD_plus_15_next'])

DPD_plus_15_next,0.0,1.0
rs2_bin,,
1,195,36
2,363,32
3,1297,57
4,1184,31
5,2088,31
